<a href="https://colab.research.google.com/github/synergit/adversarial-robustness-toolbox/blob/development_issue_1331/sign_opt_CIFAR10_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: http://home.mit.bme.hu/~hadhazi/Oktatas/NN18/dem3/html_demo/CIFAR-10Demo.html

In [2]:
# mount google drive https://medium.com/@ml_kid/how-to-save-our-model-to-google-drive-and-reuse-it-2c1028058cb2
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ls /content/gdrive

MyDrive


In [4]:
!pip install git+https://github.com/synergit/adversarial-robustness-toolbox@development_issue_1331
!pip install adversarial-robustness-toolbox

  Cloning https://github.com/synergit/adversarial-robustness-toolbox (to revision development_issue_1331) to /tmp/pip-req-build-0u5mz1m4
  Running command git clone -q https://github.com/synergit/adversarial-robustness-toolbox /tmp/pip-req-build-0u5mz1m4
  Running command git checkout -b development_issue_1331 --track origin/development_issue_1331
  Switched to a new branch 'development_issue_1331'
  Branch 'development_issue_1331' set up to track remote branch 'development_issue_1331' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 5.4 MB/s 
     |████████████████████████████████| 34.5 MB 12 kB/s 
  Created wheel for adversarial-robustness-toolbox: filename=adversarial_robustness_toolbox-1.9.1-py3-none-any.whl size=1172190 sha256=b2da1323eb93783c28e7065a21ddd870bd00852f4c79f33fe3b7dbbde01e03cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-1ur7

In [5]:
# art.utils only loads first 5000 training and 500 test data
from art.utils import load_cifar10
import matplotlib.pyplot as plt

def load_data(raw_image, length):
  (x_train, y_train), (x_test, y_test), min_, max_ = load_cifar10(raw=raw_image)#load_dataset(str("cifar10"))
  # x_train.shape (5000, 32, 32, 3)
  # y_train.shape (5000, 10)
  x_train, y_train = x_train[:length], y_train[:length]
  # x_test.shape (500, 32, 32, 3)
  # y_test.shape (500, 10)
  x_test, y_test = x_test[:int(length/10)], y_test[:int(length/10)]
  print(x_train[0].shape)
  
  imshow_CIFAR10(raw_image, x_train, y_train, 5)
  
  return (x_train, y_train), (x_test, y_test), min_, max_


def imshow_CIFAR10(raw_image, x_train, y_train, length):
  if raw_image:
    cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    print('Example training images and their labels: ' + str([x[0] for x in y_train[0:length]])) 
    print('Corresponding classes for the labels: ' + str([cifar_classes[x[0]] for x in y_train[0:length]]))

    f, axarr = plt.subplots(1, length)
    f.set_size_inches(16, 6)

    for i in range(5):
        img = x_train[i]
        axarr[i].imshow(img)
    plt.show()
  else:
    print('change `raw_image` to be True for showing images')

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

# Define the neural network model, return logits instead of activation in forward method
###
# define the network as https://arxiv.org/pdf/1608.04644.pdf, table 1
###
class Net_table1_CIFAR10(nn.Module):
    def __init__(self):
        super(Net_table1_CIFAR10, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1)
        self.conv_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1)
        self.conv_3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1)
        self.conv_4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1)
        # https://discuss.pytorch.org/t/calculation-for-the-input-to-the-fully-connected-layer/82774/11
        # x.shape: torch.Size([2, 128, 5, 5]) so, 128*5*5
        self.fc_1 = nn.Linear(in_features= 3200, out_features=256) 
        self.fc_2 = nn.Linear(in_features=256, out_features=10)
        
    # https://github.com/Carco-git/CW_Attack_on_MNIST/blob/master/MNIST_Model.py
    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv_3(x))
        x = F.relu(self.conv_4(x))
        x = F.max_pool2d(x, 2, 2)
        # print(x.shape) # torch.Size([2, 128, 5, 5])
        x = x.view(-1, 3200)
        
        x = F.relu(self.fc_1(x))
        x = F.dropout(x, p=0.5)
        x = self.fc_2(x)
        return x



In [7]:
# load the data

raw_image = False
(x_train, y_train), (x_test, y_test), min_, max_ = load_data(raw_image, 5000)
# Create the model

model_table1 = Net_table1_CIFAR10()

# model_scratch = Net_table1_CIFAR10()
# print(summary(model_scratch,(3, 32, 32)))


(32, 32, 3)
change `raw_image` to be True for showing images


In [8]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [9]:
import numpy as np
from art.estimators.classification import PyTorchClassifier
criterion = nn.CrossEntropyLoss()
optimizer_table1 = optim.SGD(model_table1.parameters(), lr=0.01, momentum=0.9)

# Create the ART classifier

classifier_table1 = PyTorchClassifier(
    model=model_table1,
    clip_values=(min_, max_),
    loss=criterion,
    optimizer=optimizer_table1,
    input_shape=(3, 32, 32),
    nb_classes=10,
    device_type = "gpu",
)

# Swap axes to PyTorch's NCHW format
x_train = np.transpose(x_train, (0, 3, 2, 1)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 2, 1)).astype(np.float32)

ML_model_Filename = "Pytorch_Model_CIFAR10_table1-0-1.pth"
path = F"/content/gdrive/MyDrive/{ML_model_Filename}"

try:
    with open(path, 'rb') as file:  
        print('Found existing model, loading the model')
        classifier_table1 = torch.load(file)
except FileNotFoundError:
    print('No existing model, training the model instead')
    classifier_table1.fit(x_train, y_train, batch_size=128, nb_epochs=50)
    # Save the model
    with open(path, 'wb'):  
        # pickle.dump(classifier_table1, file)
        torch.save(classifier_table1, path)

# print(classifier_table1) 

# Step 5: Evaluate the ART classifier on benign test examples
predictions = classifier_table1.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Found existing model, training the model instead
No existing model, training the model instead
Accuracy on benign test examples: 46.6%


In [10]:
from art.attacks.evasion.sign_opt import SignOPTAttack

q = 4000
e = 0.5
clipping = False
l = 100
attack = SignOPTAttack(estimator=classifier_table1, 
                           targeted=False, 
                           query_limit=q, 
                           eval_perform=True, verbose=False, 
                           clipped=clipping)

x_test_adv = attack.generate(x=x_test[:l], y=np.zeros(y_test.shape), x_train=x_train)


L2 = attack.logs.sum()/l
count = 0
for l2 in attack.logs:
    if l2 <=e and l2 != 0.0:
        count += 1
SR = ((count)/l)*100

print(attack.logs)
print(f'L2={L2}, SR={SR}')

predictions = classifier_table1.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:l], axis=1)) / len(y_test[:l])
# print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))
print(f'{q}, {e}, {round(L2,2)}, {SR}%, {clipping}, {accuracy*100}%')

Sign_OPT attack:   0%|          | 0/100 [00:00<?, ?it/s]

[3.63077476e-07 2.15862608e+00 3.06862053e-02 3.07669659e-04
 4.65105171e-04 8.06327735e-04 3.49948932e-05 2.14755490e-01
 5.86248934e-04 5.55742493e+01 6.07027439e-04 4.23942111e-04
 2.43718745e-07 8.58769894e+00 7.01756915e-04 6.17819722e-04
 8.10911588e-05 1.00471554e-02 2.87128240e-03 1.85938203e-04
 5.64768463e-02 5.02442941e-04 4.20015058e-05 1.74325527e-04
 5.15746418e-04 1.81431649e-03 1.12290843e-03 6.53796131e-04
 7.65279353e-01 8.35118466e-04 2.31516409e+00 9.86502945e-01
 1.96350084e-05 3.66420863e-04 5.95684385e+00 2.08417514e-05
 1.57229113e-06 3.26711535e-02 1.32883763e+00 7.96598382e-04
 2.36933905e-04 5.99858910e-02 6.26952853e-04 9.43135929e+00
 7.74046135e+00 5.65805880e-04 1.48576847e-03 7.27244013e-04
 6.52411652e+00 5.51405014e-04 1.78790942e-04 3.96190071e+00
 1.21284947e-05 8.20665248e-03 5.54088154e-04 1.08999882e+01
 3.85277438e+00 1.59264764e-05 3.54639354e-07 1.97978485e-02
 3.28643751e+00 1.10633992e-01 5.27602213e-04 3.55655036e-04
 2.11545610e+00 1.030555